## Gensim version

gensim이 버전업데이트를 최근에 자주해서, 자료의 gensim version을 미리 적어둡니다. 

현재 최신버전은 3.7 입니다. 이번 실습에서도 3.6 을 이용하였습니다.

곧 버전이 4.0+ 으로 올라갈 것 같은데, 지금까지는 Word2Vec.most_similar () 함수를 이용하여 유사어 검색이 가능했습니다. 4.0+ 이후에 이 함수가 Word2Vec.wv.most_similar 로 옮겨집니다.

In [1]:
import config
import gensim
print('Gensim version = {}'.format(gensim.__version__))

import warnings
warnings.filterwarnings('ignore')

soynlp=0.0.49
added lovit_textmining_dataset
Gensim version = 3.6.0


## Word2Vec 학습

Gensim 의 Word2Vec 을 학습하기 위해서는 한 문장이 list of str 형식인 input 이 필요합니다. 하지만 모든 리뷰들을 메모리에 올리지 않고도 학습할 수 있습니다. Generator 로 input data class 를 만든 뒤, \_\_iter\_\_ 함수를 구현하면 됩니다. 영화 평 데이터는 <영화 아이디, 영화 평, 평점> 이 tap separated 로 구분된 데이터 입니다. 이를 처리하는 부분만 아래처럼 구현합니다.

```python
for doc in f:
    idx, text, score = doc.split('\t')
    yield text.split()
```

데이터를 미리 토크나이징 해두었기 때문에 띄어쓰기 만으로 단어를 구분하여 list of str 로 yield 합니다.

또한, gensim 에는 verbose 기능이 없습니다. Input data 에 verbose 기능을 넣을 수도 있습니다.

In [2]:
class Word2VecComments:
    def __init__(self, path, verbose=False):
        self.path = path
        self.verbose = verbose
        self.n_iter = 0

    def __iter__(self):
        # <idx, texts, rates>
        with open(self.path, encoding='utf-8') as f:
            for i, doc in enumerate(f):
                if self.verbose and (i % 10000 == 0):
                    print('\riter={}, sents={} ...'.format(self.n_iter, i), end='')
                yield self._tokenize(doc)
            if self.verbose:
                print('\riter={}, sents={} done'.format(self.n_iter, i))
            self.n_iter += 1

    def _tokenize(self, doc):
        idx, text, rate = doc.strip().split('\t')
        return text.split()

In [3]:
from navermovie_comments import get_movie_comments_path

path = get_movie_comments_path(large=False, tokenize='soynlp_unsup')
word2vec_corpus = Word2VecComments(path, verbose=False)

for i, text in enumerate(word2vec_corpus):
    if i >= 3:
        break
    print(text)

['크리스', '토퍼', '놀란', '에게', '우리', '는', '놀란', '다']
['인셉션', '정말', '흥미진진', '하게', '봤', '었고', '크리스', '토퍼', '놀란', '감독님', '신작', '인터스텔라', '도', '이번', '주', '일요일', '에', '보러', '갑니다', '완전', '기대', '중']
['놀란', '이면', '무', '조건', '봐야', '된다', '왜냐하면', '모든', '작품', '을', '다', '히트', '쳤으', '니깐']


Gensim 의 Word2Vec 을 이용합니다. 미리 만들어둔 word2vec_corpus 를 Word2Vec 의 argument 로 입력합니다. default parameters 를 이용하여 Word2Vec을 학습힙니다. 

Word2Vec의 arguments 중에서 중요한 것들은 아래와 같습니다. 

- size: 단어의 임베딩 공간의 크기
- alpha: learning rate
- window: 한 단어의 좌/우의 문맥 크기
- min_count: 모델이 학습할 단어의 최소 출현 빈도수
- max_vocab_size: None이 아닌 숫자를 입력하면 빈도수 기준으로 상위 max_vocab_size 개수만큼의 단어만 학습
- workers: num of threads
- sg: 1이면 skipgram 이용
- negative: negative sampling에서 negative sample의 개수

In [4]:
%%time

from gensim.models import Word2Vec

word2vec_corpus.verbose = True
word2vec_model = Word2Vec(word2vec_corpus)

iter=0, sents=294492 done
iter=1, sents=294492 done
iter=2, sents=294492 done
iter=3, sents=294492 done
iter=4, sents=294492 done
iter=5, sents=294492 done
CPU times: user 1min 4s, sys: 484 ms, total: 1min 4s
Wall time: 32.2 s


학습된 Word2Vec 모델의 most_similar(단어, topn) 함수는 입력된 단어에 대하여 가장 비슷한 topn개의 다른 단어들과 유사도를 출력합니다. 

In [5]:
word2vec_model.wv.most_similar('영화')

[('sf영화', 0.6271036863327026),
 ('작품', 0.5569660663604736),
 ('영화들', 0.5210461616516113),
 ('명화', 0.5111042261123657),
 ('영화지만', 0.5031603574752808),
 ('듯하다', 0.4901500344276428),
 ('영화라서', 0.4832218289375305),
 ('것같다', 0.47459280490875244),
 ('영화인데', 0.4547157883644104),
 ('장르', 0.45321589708328247)]

In [6]:
word2vec_model.wv.most_similar('2d')

[('4d', 0.9180201292037964),
 ('2D', 0.9043574333190918),
 ('4D', 0.8830397129058838),
 ('3d', 0.8714023232460022),
 ('디지털', 0.8710218667984009),
 ('쓰리디', 0.8693499565124512),
 ('4DX', 0.8601016998291016),
 ('4dx', 0.8582479953765869),
 ('VOD', 0.8573418855667114),
 ('디비디', 0.8496630191802979)]

wv.vocab 은 {str:Vocab} 형식의 dict 입니다.

In [7]:
print(word2vec_model.wv.vocab['영화'])

Vocab(count:109144, index:0, sample_int:923530540)


Vocab 은 namedtuple 이기 때문에 count 를 가져올 수 있습니다. index 는 각 단어의 row number 입니다.

In [8]:
word2vec_model.wv.vocab['영화'].count

109144

데이터의 양이 작아 embedding 이 잘 학습되지 않았습니다.

In [9]:
len(word2vec_model.wv.vocab)

26809

큰 데이터로 미리 학습해둔 모델을 데이터에 넣어뒀습니다.

In [10]:
from navermovie_comments import load_trained_embedding

word2vec_model = load_trained_embedding(
    data_name = 'large',
    tokenize = 'soynlp_unsup',
    embedding = 'word2vec'
)

충분한 데이터로 학습하면 유사어도 잘 학습됩니다. 오탈자들도 유사어로 학습되었습니다.

In [11]:
word2vec_model.wv.most_similar('영화', topn=30)

[('애니', 0.7622032761573792),
 ('영회', 0.7138155102729797),
 ('애니메이션', 0.6826601028442383),
 ('영하', 0.6804906129837036),
 ('여화', 0.6422344446182251),
 ('영호ㅏ', 0.6399882435798645),
 ('명화', 0.6227416396141052),
 ('영화ㅋ', 0.6182453632354736),
 ('드라마', 0.6110595464706421),
 ('sf영화', 0.6031010746955872),
 ('영화들', 0.601367175579071),
 ('작품', 0.6008715629577637),
 ('장르영화', 0.5893300175666809),
 ('영화였고', 0.5842821598052979),
 ('영화내요', 0.5827130079269409),
 ('영화였네요', 0.5813204050064087),
 ('양화', 0.5806300640106201),
 ('영화인데', 0.572135329246521),
 ('애니매이션', 0.5695532560348511),
 ('에니메이션', 0.5671166181564331),
 ('영화인것', 0.5665518045425415),
 ('영화지만', 0.5654911994934082),
 ('영화ㅋㅋ', 0.5525494813919067),
 ('영환데', 0.5350443124771118),
 ('엉화', 0.5329785346984863),
 ('대중영화', 0.5315350294113159),
 ('블록버스터', 0.5307241678237915),
 ('영화ㅠㅠ', 0.5289486646652222),
 ('연화', 0.5288727879524231),
 ('경우', 0.5171139240264893)]

1점과 유사한 단어가 한글로 쓴 '일점', 그 이후로는 1, 2, 3, ... 이렇게 점수가 멀어져가는 것도 볼 수 있습니다

'십점' 이라는 말은 점수가 들어갈 수 있는 문맥에서 나오는 말이기도 하지만, 긍정적인 표현에서 더 많이 나왔을 것입니다. 그렇기 때문에 1점 보다도, 천점, 백점 같은 단어들이 더 유사하게 학습됩니다 (네이버 영화에서 10점은 별 다섯개입니다)

평론가는 평론가, 평론 단체 (씨네21 등)끼리 뭉쳐 나오는 걸 볼 수 있습니다

또한 Word2Vec 은 단어가 영어, 숫자, 한글인지 전혀 중요하지 않습니다. 단어가 함께 섞여도 됩니다.

In [12]:
for word in '하정우 1점 십점 이동진 평론가 평론 스토리 조연 배우 포디 4d 영등포 롯시 ocn'.split():
    similar_words = word2vec_model.wv.most_similar(word)
    similar_words = [word for word, sim in similar_words]
    print('similar({}) = {}'.format(
        word, ' '.join(similar_words)))

similar(하정우) = 송강호 이정재 공유 황정민 이병헌 손현주 이범수 주진모 유해진 톰크루즈
similar(1점) = 일점 별한개 2점 3점 4점 최하점 십점 5점 별하나 구점
similar(십점) = 일점 구점 백점 1점 반점 만점 11점 최하점 삼점 천점
similar(이동진) = 박평식 이용철 송경원 김현수 씨네21 평론가님 황진미 기자님 기자 김봉석
similar(평론가) = 전문가 기자 씨네21 평론가들 전문가들 박평식 황진미 기자들 한겨례 이동진
similar(평론) = 비평 평 평론가 악평 논평 평가 아는척 한줄평 전문가 댓
similar(스토리) = 줄거리 시나리오 내용 이야기전개 영화전개 내러티브 스로리 플롯 소재 사건전개
similar(조연) = 주조연 단역 조연들 주연 명품조연 엑스트라 연기자 명품조연들 박희순 김인권
similar(배우) = 연기자 배우들 연기자들 베우 한석규 조연 조연들 하정우 김태리 스타들
similar(포디) = 4디 4d 쓰리디 3디 투디 4D 4dx 4DX 삼디 아이맥스3D
similar(4d) = 4D 3d 포디 4DX 쓰리디 3D 4dx 아이맥스 2d 3디
similar(영등포) = 코엑스 용산 왕십리 메가박스 신촌 상암 씨지비 일산 김포공항 프리머스
similar(롯시) = 영등포 코엑스 용산 상암 일산 메가박스 신촌 씨지비 프리머스 김포공항
similar(ocn) = OCN 오씨엔 Tv 케이블 Ocn 공중파 일반관 출발비디오여행 tv 스타리움


Word2Vec model의 단어 벡터들은 Word2Vec.wv 아래에 저장되어 있습니다. gensim version 1.x 부터 Word2Vec.wv에서 단어 벡터를 따로 관리합니다.

`Word2Vec.wv.syn0`: 실제 단어가 저장되어 있는 행렬로, numpy.ndarray 형태입니다.

```python
word2vec_model.wv.vectors.shape # (93234, 100)
```

`word2vec_model.wv.vectors_norm` : cosine similarity 를 위하여 row normalize 를 한 행렬로 모양은 같지만, 벡터의 2 norm 이 1 입 니다. 

```python
word2vec_model.wv.vectors_norm.shape # (93234, 100)
sum(word2vec_model.wv.vectors[0] **2) # 318.1980813240516
sum(word2vec_model.wv.vectors_norm[0] **2) # 1.0000001593821253
```

Word2Vec.wv.index2word는 단어별 index가 저장되어 있습니다. 

In [13]:
print(len(word2vec_model.wv.index2word))
print(type(word2vec_model.wv.index2word))
print(word2vec_model.wv.index2word[0])

93234
<class 'list'>
영화


## Doc2Vec 학습

Doc2Vec을 학습하기 위해서는 각각 문서의 label이 저장되어야 합니다. 이를 위하여 TaggedDocument라는 클래스가 이용됩니다. TaggedDocument는 단어들을 words에, 레이블 정보를 tags에 리스트 형태로 입력합니다

In [14]:
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

class Doc2VecComments(Word2VecComments):
    def _tokenize(self, doc):
        idx, text, rate = doc.strip().split('\t')
        return TaggedDocument(
                    words=text.split(), tags=['#%s' % idx]
                )

path = get_movie_comments_path(large='large', tokenize='soynlp_unsup')
doc2vec_corpus = Doc2VecComments(path)

for i, doc in enumerate(doc2vec_corpus):
    if i > 3: break
    print(doc)

TaggedDocument(['명불허전'], ['#72523'])
TaggedDocument(['왠지', '고사', '피의', '중간', '고사', '보다', '재미', '가', '없을듯', '해요', '만약', '보게', '된다면', '실망', '할듯'], ['#72523'])
TaggedDocument(['티아라', '사랑', '해', 'ㅜ'], ['#72523'])
TaggedDocument(['황정음', '윤시윤', '지붕킥', '인연', '김수로', '티아라', '지연', '공부의신', '인연', '너무', '너무', '재미', '있어요'], ['#72523'])


In [15]:
TRAIN = False
if TRAIN:
    doc2vec_model = Doc2Vec(doc2vec_corpus)
else:
    doc2vec_model = load_trained_embedding(
        data_name='large', tokenize='soynlp_unsup', embedding='doc2vec')

Doc2Vec 에서도 유사 단어 검색이 가능합니다.

In [16]:
doc2vec_model.wv.most_similar('영화', topn=10)

[('애니', 0.7179281115531921),
 ('영회', 0.6447778940200806),
 ('여화', 0.6053318977355957),
 ('영하', 0.592547595500946),
 ('엉화', 0.5851074457168579),
 ('애니메이션', 0.5821044445037842),
 ('sf영화', 0.581710696220398),
 ('양화', 0.5816899538040161),
 ('애니영화', 0.5751307010650635),
 ('영화였고', 0.5723588466644287)]

In [17]:
doc2vec_model.wv.most_similar('디카프리오', topn=10)

[('톰하디', 0.8216360807418823),
 ('레오', 0.8157205581665039),
 ('브래드피트', 0.8099828958511353),
 ('니콜라스홀트', 0.8012720346450806),
 ('베네딕트', 0.8000676035881042),
 ('앤해서웨이', 0.7960273027420044),
 ('앤헤서웨이', 0.7938777208328247),
 ('로다주', 0.7688084840774536),
 ('컴버배치', 0.76687091588974),
 ('프레디', 0.7544741630554199)]

Doc2Vec model의 .docvecs안에는 document vector와 관련된 정보들이 저장되어 있습니다

In [18]:
len(doc2vec_model.docvecs)

172

In [19]:
type(doc2vec_model.docvecs)

gensim.models.keyedvectors.Doc2VecKeyedVectors

doctags 에 들어있는 offset 은 document vector 의 임베딩 메트릭스의 row id 이며, word_count 는 각 태그에 해당하는 문서에 단어가 몇 개 있었는지, doc_count 는 각 태그에 해당하는 문서가 몇 번 등장하였는지입니다

In [20]:
doctags = doc2vec_model.docvecs.doctags.items()
doctags = sorted(doctags, key=lambda x:x[1].offset)
doctags[:3]

[('#72523', Doctag(offset=0, word_count=89878, doc_count=10187)),
 ('#59845', Doctag(offset=1, word_count=139795, doc_count=13095)),
 ('#109753', Doctag(offset=2, word_count=200116, doc_count=10361))]

`#72523` 은 offset=0 은 docvec 에서의 row id 가 0 라는 의미입니다. 

In [21]:
doc2vec_model.docvecs.most_similar('#72523')

[('#73344', 0.8099367618560791),
 ('#48246', 0.7621855735778809),
 ('#42589', 0.7321667671203613),
 ('#102824', 0.729533314704895),
 ('#76080', 0.7153506278991699),
 ('#88225', 0.7075489163398743),
 ('#48227', 0.7025582790374756),
 ('#41450', 0.6783559918403625),
 ('#64191', 0.676222026348114),
 ('#75355', 0.6696109771728516)]

Document vector의 row id로도 most_similar를 찾을 수 있습니다. 

In [22]:
doc2vec_model.docvecs.most_similar(0)

[('#73344', 0.8099367618560791),
 ('#48246', 0.7621855735778809),
 ('#42589', 0.7321667671203613),
 ('#102824', 0.729533314704895),
 ('#76080', 0.7153506278991699),
 ('#88225', 0.7075489163398743),
 ('#48227', 0.7025582790374756),
 ('#41450', 0.6783559918403625),
 ('#64191', 0.676222026348114),
 ('#75355', 0.6696109771728516)]

## Doc2Vec 해석하기

In [23]:
from navermovie_comments import load_id_to_movie

idx_to_movie = load_id_to_movie()

영화 아이디를 영화 제목으로 바꿔서 해석해봅시다

영화 리뷰를 기준으로 각 영화를 document vector로 표현하였을 때 라라랜드와 리뷰가 비슷한 영화는 '비긴 어게인', '어바웃 타임' 등입니다

In [24]:
def as_name(similar):
    idx = similar[0][1:]
    return (idx_to_movie.get(idx, 'unknown'), idx, similar[1])


print('라라랜드\n')

for similar in doc2vec_model.docvecs.most_similar('#134963'):
    print(as_name(similar))

라라랜드

('비긴 어게인', '96379', 0.9024428129196167)
('어바웃 타임', '92075', 0.8169595003128052)
('인턴', '118917', 0.710594654083252)
('인사이드 아웃', '115622', 0.6986126899719238)
('시간을 달리는 소녀', '63513', 0.6862818002700806)
('하울의 움직이는 성', '39640', 0.6862198114395142)
('레미제라블', '89755', 0.6842658519744873)
('미스 페레그린과 이상한 아이들의 집', '129383', 0.6826176047325134)
('뷰티 인사이드', '129050', 0.6816157698631287)
('겨울왕국', '100931', 0.6804711222648621)


In [25]:
print('관상\n')
for similar in doc2vec_model.docvecs.most_similar('#93728'):
    print(as_name(similar))

관상

('역린', '108225', 0.8302776217460632)
('광해, 왕이 된 남자', '83893', 0.8152835369110107)
('군도:민란의 시대', '99752', 0.7720687389373779)
('사도', '121922', 0.7519895434379578)
('도둑들', '78726', 0.7223831415176392)
('검사외전', '130903', 0.719760537147522)
('신세계', '91031', 0.7194761037826538)
('의형제', '52548', 0.7159029245376587)
('밀정', '137952', 0.6845557689666748)
('쌍화점', '45232', 0.6726131439208984)


In [26]:
print('광해 왕이된 남자\n')
for similar in doc2vec_model.docvecs.most_similar('#83893'):
    print(as_name(similar))

광해 왕이된 남자

('관상', '93728', 0.8152835369110107)
('의형제', '52548', 0.7779906988143921)
('라디오 스타', '58088', 0.7411540746688843)
('파파로티', '85640', 0.7076984643936157)
('수상한 그녀', '107924', 0.6803666353225708)
('시라노; 연애조작단', '73318', 0.6777384281158447)
('왕의 남자', '39894', 0.6738080382347107)
('미녀는 괴로워', '39157', 0.6719443798065186)
('도둑들', '78726', 0.6672611236572266)
('신세계', '91031', 0.6661689281463623)


In [27]:
print('아바타\n')
for similar in doc2vec_model.docvecs.most_similar('#62266'):
    print(as_name(similar))

아바타

('트랜스포머', '61521', 0.8492670059204102)
('디스트릭트 9', '64129', 0.8323827385902405)
('2012', '49727', 0.7791959643363953)
('인셉션', '52515', 0.7713688015937805)
('스카이라인', '76581', 0.7526602149009705)
('다크 나이트', '62586', 0.746321439743042)
('그래비티', '47370', 0.738884687423706)
('퍼시픽 림', '86867', 0.7356865406036377)
('리얼 스틸', '76460', 0.7262969017028809)
('타이타닉', '18847', 0.720104455947876)
